## Web Search Test
The **Web Search Test** ant demonstrates the use of a Web API secured using an API key with an auth header. This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [7]:
#r "nuget: AntRunnerLib, 0.7.3"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.7.3

## Ensure the Web Search Test assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search Test" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search Test" contains the following files:
```
│   instructions.md - The instructions
│   manifest.json - The base assistant definition
│
└───OpenAPI
        api.bing.microsoft.com.json - A swagger file defining a tool named 'crawl' that uses web search
        auth.json - See below... 
```        

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [8]:
var assistantId = await AssistantUtility.Create("Web Search Test", config);
Console.WriteLine(assistantId)

asst_uf8WMAeEufFt5akAYMsT14kw


## Run the Assistant

`output.Dialog` shows the conversation.

In [23]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search Test",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer. You can perform multiple searches if necessary",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer. You can perform multiple searches if necessary

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()
# Calculate the upcoming weekend dates (Saturday and Sunday)
this_saturday = today + timedelta((5-today.weekday()) % 7)
this_sunday = this_saturday + timedelta(1)

# Format dates for display
today_str = today.strftime('%Y-%m-%d')
this_saturday_str = this_saturday.strftime('%Y-%m-%d')
this_sunday_str = this_sunday.strftime('%Y-%m-%d')

today_str, this_saturday_str, this_sunday_str
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta on September 28, 2024", "answerCount": 5, "count": 5, "textFormat": "HTML"}
```
and got this result:
```
{"_type": "SearchResponse", "queryContext": {"originalQuery": "events in Atlanta on September 28, 2024", "askUserForLocation": true}, "webPages": {"webSearchUrl": "https:\/\/www.bing.com\/search?q=events+in+Atlanta+on+September+28%2c+2024", "totalEstimatedMatches": 1510000, "value": [{"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0", "name": "Atlanta Events September 2024: Concerts, Shows, Sports...", "url": "https:\/\/atlanta-ga.events\/september\/", "datePublished": "2024-09-19T00:00:00.0000000", "datePublishedFreshnessText": "3 days ago", "isFamilyFriendly": true, "displayUrl": "https:\/\/atlanta-ga.events\/september", "snippet": "Check Out The Atlanta Events Calendar for September 2024. See the List Of All Current &amp; Upcoming Events at the Lowest Possible Price. ... Prices from $28 Avg. price ~ $64. 18 tickets remaining! Tickets; Sep. 21. 2024. 8:00 PM Sat. Mat Kearney. The Tabernacle - GA | Capacity: 2600. 30303, 152 Luckie St, Atlanta, GA, US Prices from $37 Avg. price ...", "dateLastCrawled": "2024-09-19T19:10:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=events+in+Atlanta+on+September+28%2c+2024&d=4854189675467386&mkt=en-US&setlang=en-US&w=ShlkdiIwTMefbkmdL1lCvH4mS1bjztOY", "language": "en", "isNavigational": true, "noCache": false, "siteName": "Atlanta Events"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1", "name": "Atlanta Events - September 2024 | Go South Atlanta", "url": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-september.html", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4&w=80&h=80&c=1&pid=5.1", "isFamilyFriendly": true, "displayUrl": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-september.html", "snippet": "More details. – Atlanta Black Pride Weekend, August 28 - September 2 2024. Annual LGBTQ+ festival with male, female and youth events across Atlanta, including parties, block parties, food events, performances, film festival, and more. More details. – Dragon Con, August 29 - September 2 2024.", "dateLastCrawled": "2024-09-21T02:18:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 248, "imageId": "OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=events+in+Atlanta+on+September+28%2c+2024&d=4712253893115949&mkt=en-US&setlang=en-US&w=D6V2DS_RjZRe4oOvzq1dGpeg_GE4sHjt", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Go South Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2", "name": "Cool Things To Do in September in Atlanta", "url": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-september-in-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-08-23T00:00:00.0000000", "datePublishedDisplayText": "Aug 23, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-september-in-atlanta", "snippet": "The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta’s most iconic streets, offers fun-filled entertainment, live music — everything from R&amp;B to gospel to alternative favorites — international food court, national artists, celebrities and vendors of all types.", "dateLastCrawled": "2024-09-20T02:07:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 248, "imageId": "OIP.m0ENCio7ytS2o9PQRWDCRgHaD4"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=events+in+Atlanta+on+September+28%2c+2024&d=4692574351720481&mkt=en-US&setlang=en-US&w=IyKVbCksY9VV74GkIkdoRE4k4vk9MNbJ", "language": "en", "isNavigational": false, "richFacts": [{"label": {"text": "Author"}, "items": [{"text": "Mary Welch"}], "hint": {"text": "BASE:AUTHOR"}}], "noCache": false, "siteName": "Discover Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3", "name": "80+ fall festivals &amp; fairs around metro Atlanta: Fun things to do in ...", "url": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-14T00:00:00.0000000", "datePublishedDisplayText": "Sep 14, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta", "snippet": "September 28, 2024 East Atlanta Village FREE. This iconic street festival celebrates East Atlanta’s culture, art, and quirkiness! ... An annual October event! Atlanta Pride features a festival and parade in Midtown, plus a wide variety of associated events around town. Details for Atlanta Pride.", "dateLastCrawled": "2024-09-21T00:01:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 277, "imageId": "OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=events+in+Atlanta+on+September+28%2c+2024&d=5012794228343791&mkt=en-US&setlang=en-US&w=rgc8005sQAPnjjGt9vluoyq1DPm6ofe-", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Atlanta on the Cheap"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4", "name": "Atlanta Festival Calendar: 2024", "url": "https:\/\/unexpectedatlanta.com\/atlanta-festival-calendar-2024\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-05-02T00:00:00.0000000", "datePublishedDisplayText": "May 2, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/unexpectedatlanta.com\/atlanta-festival-calendar-2024", "snippet": "Atlanta Jazz Festival – May 25-27, 2024. Held in Piedmont Park, the Atlanta Jazz Festival is an annual celebration of music, culture, and the art of Jazz. Free to all who join, this Memorial Day Weekend event draws fans, professional musicians, and everyone in between, for a 3-day-long festival showcasing 15+ artists.", "dateLastCrawled": "2024-09-20T03:02:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 474, "imageId": "OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=events+in+Atlanta+on+September+28%2c+2024&d=4520195830737555&mkt=en-US&setlang=en-US&w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Unexpected Atlanta"}]}, "relatedSearches": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches", "value": [{"text": "atlanta events september 2024", "displayText": "atlanta events september 2024", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+events+september+2024"}, {"text": "atlanta in september 2023", "displayText": "atlanta in september 2023", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+in+september+2023"}, {"text": "festivals in atlanta 2024", "displayText": "festivals in atlanta 2024", "webSearchUrl": "https:\/\/www.bing.com\/search?q=festivals+in+Atlanta+2024"}, {"text": "atlanta fall festivals 2023", "displayText": "atlanta fall festivals 2023", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+fall+festivals+2023"}, {"text": "september events in atlanta", "displayText": "september events in atlanta", "webSearchUrl": "https:\/\/www.bing.com\/search?q=september+events+in+Atlanta"}, {"text": "atlanta in september", "displayText": "atlanta in september", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+in+september"}, {"text": "underground atlanta september events", "displayText": "underground atlanta september events", "webSearchUrl": "https:\/\/www.bing.com\/search?q=underground+Atlanta+september+events"}, {"text": "events in atlanta this weekend", "displayText": "events in atlanta this weekend", "webSearchUrl": "https:\/\/www.bing.com\/search?q=events+in+Atlanta+this+weekend"}]}, "rankingResponse": {"mainline": {"items": [{"answerType": "WebPages", "resultIndex": 0, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0"}}, {"answerType": "WebPages", "resultIndex": 1, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1"}}, {"answerType": "WebPages", "resultIndex": 2, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2"}}, {"answerType": "WebPages", "resultIndex": 3, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3"}}, {"answerType": "WebPages", "resultIndex": 4, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4"}}, {"answerType": "RelatedSearches", "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches"}}]}}}
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta on September 29, 2024", "answerCount": 5, "count": 5, "textFormat": "HTML"}
```
and got this result:
```
{"_type": "SearchResponse", "queryContext": {"originalQuery": "events in Atlanta on September 29, 2024", "askUserForLocation": true}, "webPages": {"webSearchUrl": "https:\/\/www.bing.com\/search?q=events+in+Atlanta+on+September+29%2c+2024", "totalEstimatedMatches": 2010000, "value": [{"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0", "name": "Atlanta Events September 2024: Concerts, Shows, Sports...", "url": "https:\/\/atlanta-ga.events\/september\/", "datePublished": "2024-09-19T00:00:00.0000000", "datePublishedFreshnessText": "3 days ago", "isFamilyFriendly": true, "displayUrl": "https:\/\/atlanta-ga.events\/september", "snippet": "Terminal West At King Plow Arts Center. Sep 20, 2024 8:00 PM. Montell Fish. Upcoming Events: 2. Buckhead Theatre. Sep 20, 2024 8:00 PM. Show More. Check Out The Atlanta Events Calendar for September 2024. See the List Of All Current &amp; Upcoming Events at the Lowest Possible Price.", "dateLastCrawled": "2024-09-19T19:10:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=events+in+Atlanta+on+September+29%2c+2024&d=4854189675467386&mkt=en-US&setlang=en-US&w=ShlkdiIwTMefbkmdL1lCvH4mS1bjztOY", "language": "en", "isNavigational": true, "noCache": false, "siteName": "Atlanta Events"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1", "name": "Atlanta Events - September 2024 | Go South Atlanta", "url": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-september.html", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4&w=80&h=80&c=1&pid=5.1", "isFamilyFriendly": true, "displayUrl": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-september.html", "snippet": "More details. – Dragon Con, August 29 - September 2 2024. One of the biggest annual pop culture conventions in the country. More details. – Atlanta Black Theatre Festival, August 30 - September 1 2024. Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events.", "dateLastCrawled": "2024-09-21T20:12:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 248, "imageId": "OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=events+in+Atlanta+on+September+29%2c+2024&d=4712253893115949&mkt=en-US&setlang=en-US&w=D6V2DS_RjZRe4oOvzq1dGpeg_GE4sHjt", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Go South Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2", "name": "Cool Things To Do in September in Atlanta", "url": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-september-in-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-08-23T00:00:00.0000000", "datePublishedDisplayText": "Aug 23, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-september-in-atlanta", "snippet": "The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta’s most iconic streets, offers fun-filled entertainment, live music — everything from R&amp;B to gospel to alternative favorites — international food court, national artists, celebrities and vendors of all types.", "dateLastCrawled": "2024-09-20T02:07:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 248, "imageId": "OIP.m0ENCio7ytS2o9PQRWDCRgHaD4"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=events+in+Atlanta+on+September+29%2c+2024&d=4692574351720481&mkt=en-US&setlang=en-US&w=IyKVbCksY9VV74GkIkdoRE4k4vk9MNbJ", "language": "en", "isNavigational": false, "richFacts": [{"label": {"text": "Author"}, "items": [{"text": "Mary Welch"}], "hint": {"text": "BASE:AUTHOR"}}], "noCache": false, "siteName": "Discover Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3", "name": "Events &amp; Things To Do in Atlanta In September 2024 | AllEvents.in", "url": "https:\/\/allevents.in\/atlanta\/september", "isFamilyFriendly": true, "displayUrl": "https:\/\/allevents.in\/atlanta\/september", "snippet": "Here’s your guide to all the happening concerts, festivals and events in September 2024 in Atlanta. Catch the best of everything that your city has to offer here. Right from the flea market events to all the festivals; the September 2024 events in Atlanta is all curated and sorted for you. Get going, buy your tickets and register to all the ...", "dateLastCrawled": "2024-09-17T21:07:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=events+in+Atlanta+on+September+29%2c+2024&d=4505378191124551&mkt=en-US&setlang=en-US&w=tSwBY_ub0aUAzxJYI-i48UowFfDohTmv", "language": "en", "isNavigational": false, "noCache": false, "siteName": "AllEvents.in"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4", "name": "80+ fall festivals &amp; fairs around metro Atlanta: Fun things to do in ...", "url": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-14T00:00:00.0000000", "datePublishedDisplayText": "Sep 14, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta", "snippet": "September 29, 2024 Oakland Cemetery Atlanta. Oakland Cemetery’s beloved fall festival is based on the Victorian tradition of spending a day of leisure in the cemetery. It has always been a free event — however, in 2023, it’s being combined with Tunes from the Tombs and is a ticketed event. Details for this event", "dateLastCrawled": "2024-09-21T00:01:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 277, "imageId": "OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=events+in+Atlanta+on+September+29%2c+2024&d=5012794228343791&mkt=en-US&setlang=en-US&w=rgc8005sQAPnjjGt9vluoyq1DPm6ofe-", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Atlanta on the Cheap"}], "someResultsRemoved": true}, "relatedSearches": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches", "value": [{"text": "atlanta events september 2024", "displayText": "atlanta events september 2024", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+events+september+2024"}, {"text": "atlanta in september 2023", "displayText": "atlanta in september 2023", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+in+september+2023"}, {"text": "atlanta airport events september 2024", "displayText": "atlanta airport events september 2024", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+airport+events+september+2024"}, {"text": "september events in atlanta", "displayText": "september events in atlanta", "webSearchUrl": "https:\/\/www.bing.com\/search?q=september+events+in+Atlanta"}, {"text": "underground atlanta september events", "displayText": "underground atlanta september events", "webSearchUrl": "https:\/\/www.bing.com\/search?q=underground+Atlanta+september+events"}, {"text": "atlanta in september", "displayText": "atlanta in september", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+in+september"}, {"text": "events in atlanta this weekend", "displayText": "events in atlanta this weekend", "webSearchUrl": "https:\/\/www.bing.com\/search?q=events+in+Atlanta+this+weekend"}, {"text": "things to do in atlanta in september", "displayText": "things to do in atlanta in september", "webSearchUrl": "https:\/\/www.bing.com\/search?q=things+to+do+in+Atlanta+in+september"}]}, "rankingResponse": {"mainline": {"items": [{"answerType": "WebPages", "resultIndex": 0, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0"}}, {"answerType": "WebPages", "resultIndex": 1, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1"}}, {"answerType": "WebPages", "resultIndex": 2, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2"}}, {"answerType": "WebPages", "resultIndex": 3, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3"}}, {"answerType": "WebPages", "resultIndex": 4, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4"}}, {"answerType": "RelatedSearches", "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches"}}]}}}
```

Here are some events happening in Atlanta this weekend, on September 28 and 29, 2024:

### Saturday, September 28, 2024
1. **Sweet Auburn Music Fest**
   - Location: Auburn Avenue
   - Description: Enjoy live music ranging from R&B to gospel, an international food court, national artists, celebrities, and various vendors.
   - [More details](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

2. **East Atlanta Village Fall Festival**
   - Location: East Atlanta Village
   - Description: Celebrate East Atlanta's culture, art, and quirkiness with a free street festival.
   - [More details](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

3. **Mat Kearney Concert**
   - Venue: The Tabernacle - GA
   - Description: Enjoy a live performance by Mat Kearney.
   - [More details](https://atlanta-ga.events/september/)

### Sunday, September 29, 2024
1. **Sweet Auburn Music Fest**
   - Location: Auburn Avenue
   - Description: The festival continues with more live music, food, and entertainment.
   - [More details](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)

2. **Oakland Cemetery Fall Festival**
   - Location: Oakland Cemetery
   - Description: Enjoy a Victorian tradition of spending a day in the cemetery combined with Tunes from the Tombs, making it a ticketed event.
   - [More details](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

For a full list of events and more details, you can visit:
- [Atlanta Events September 2024](https://atlanta-ga.events/september/)
- [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)
- [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
- [Atlanta on the Cheap](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
- [Unexpected Atlanta Festival Calendar](https://unexpectedatlanta.com/atlanta-festival-calendar-2024/)

Enjoy your weekend in Atlanta!


In [21]:
output.LastMessage.DisplayAs("text/markdown");

Here are some interesting events and activities in Atlanta for the next four weekends:

### September 21, 2024
1. [Adair Park Festival](https://www.atlantaonthecheap.com/fall-festivals-atlanta/): A charming neighborhood festival with plenty of pie, a beer garden, artist market, kid activities, and more. Admission is free, but a pie-tasting pass costs $10.
2. [Indie Craft Experience Fall Market](https://www.fox5atlanta.com/news/things-do-weekend-metro-atlanta-sept-20-22-2024): A one-day craft and vintage market featuring more than 60 vendors at Yaarab Shrine Center.
3. [JapanFest Atlanta](https://discoveratlanta.com/events/main/): A celebration of Japanese culture at the Infinite Energy Center, featuring food, performances, and cultural exhibits.

### September 28, 2024
1. [Sweet Auburn Music Fest](https://discoveratlanta.com/events/main/): Located on Auburn Avenue, this festival offers live music from R&B to gospel, an international food court, national artists, celebrities, and vendors of all types.
2. [Carter Presidential Library & Museum](https://www.atlantaonthecheap.com/september-things-to-do-atlanta/): Free admission in honor of President Jimmy Carter's 100th birthday.
3. [Atlanta Streets Alive](https://www.atlantaonthecheap.com/fall-festivals-atlanta/): Streets temporarily closed to traffic for walking, running, biking, and other activities. 

### October 5, 2024
(Please note that no specific events for this date were found. Here are some ongoing activities you might consider.)
1. [Grant Park Past & Future Food Walk](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/): A fun walking tour showcasing the best of Atlanta's food and history.
2. [Atlanta Food & Wine Festival](https://www.experism.com/atlanta/september): Highlights the best food, wine, beer, and cocktails Atlanta has to offer.

### October 12, 2024
(Please note that no specific events for this date were found. Here are some ongoing activities you might consider.)
1. [Explore Atlanta's Fall Festivals](https://www.atlantaonthecheap.com/fall-festivals-atlanta/): Various fall festivals and fairs around metro Atlanta.
2. [Experience Atlanta's Best Eats](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/): Try some of the best eats in Atlanta on various food tours.

For more details on each event, you can visit the provided links. Enjoy your time in Atlanta!

In [22]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search Test",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.today()

# Calculate the dates for the next four weekends (Saturdays)
weekends = [today + timedelta(days=(5-today.weekday() + 7*i)) for i in range(4)]
weekends
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on September 21, 2024", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"_type": "SearchResponse", "queryContext": {"originalQuery": "things to do in Atlanta on September 21, 2024", "askUserForLocation": true}, "webPages": {"webSearchUrl": "https:\/\/www.bing.com\/search?q=things+to+do+in+Atlanta+on+September+21%2c+2024", "totalEstimatedMatches": 4650000, "value": [{"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0", "name": "Atlanta Events September 2024: Concerts, Shows, Sports...", "url": "https:\/\/atlanta-ga.events\/september\/", "datePublished": "2024-09-19T00:00:00.0000000", "datePublishedFreshnessText": "3 days ago", "isFamilyFriendly": true, "displayUrl": "https:\/\/atlanta-ga.events\/september", "snippet": "Terminal West At King Plow Arts Center. Sep 20, 2024 8:00 PM. Montell Fish. Upcoming Events: 2. Buckhead Theatre. Sep 20, 2024 8:00 PM. Show More. Check Out The Atlanta Events Calendar for September 2024. See the List Of All Current &amp; Upcoming Events at the Lowest Possible Price.", "dateLastCrawled": "2024-09-19T19:10:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+21%2c+2024&d=4854189675467386&mkt=en-US&setlang=en-US&w=ShlkdiIwTMefbkmdL1lCvH4mS1bjztOY", "language": "en", "isNavigational": true, "noCache": false, "siteName": "Atlanta Events"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1", "name": "Cool Things To Do in September in Atlanta", "url": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-september-in-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-08-23T00:00:00.0000000", "datePublishedDisplayText": "Aug 23, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-september-in-atlanta", "snippet": "The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta’s most iconic streets, offers fun-filled entertainment, live music — everything from R&amp;B to gospel to alternative favorites — international food court, national artists, celebrities and vendors of all types.", "dateLastCrawled": "2024-09-20T02:07:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 248, "imageId": "OIP.m0ENCio7ytS2o9PQRWDCRgHaD4"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+21%2c+2024&d=4692574351720481&mkt=en-US&setlang=en-US&w=IyKVbCksY9VV74GkIkdoRE4k4vk9MNbJ", "language": "en", "isNavigational": false, "richFacts": [{"label": {"text": "Author"}, "items": [{"text": "Mary Welch"}], "hint": {"text": "BASE:AUTHOR"}}], "noCache": false, "siteName": "Discover Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2", "name": "Atlanta Events - September 2024 - Go South Atlanta", "url": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-september.html", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4&w=80&h=80&c=1&pid=5.1", "isFamilyFriendly": true, "displayUrl": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-september.html", "snippet": "More details. – Dragon Con, August 29 - September 2 2024. One of the biggest annual pop culture conventions in the country. More details. – Atlanta Black Theatre Festival, August 30 - September 1 2024. Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events.", "dateLastCrawled": "2024-09-21T20:12:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 248, "imageId": "OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+21%2c+2024&d=4712253893115949&mkt=en-US&setlang=en-US&w=D6V2DS_RjZRe4oOvzq1dGpeg_GE4sHjt", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Go South Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3", "name": "80+ fall festivals &amp; fairs around metro Atlanta: Fun things to do in ...", "url": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-14T00:00:00.0000000", "datePublishedDisplayText": "Sep 14, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta", "snippet": "September 21, 2024 Adair Park Atlanta. A charming neighborhood festival with plenty of pie — as well as a beer garden, artist market, kid activities and more. Admission is free, but a pie-tasting pass costs $10. Go to the website. ... September 21, 2024 Chamblee City Hall FREE.", "dateLastCrawled": "2024-09-21T00:01:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 277, "imageId": "OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+21%2c+2024&d=5012794228343791&mkt=en-US&setlang=en-US&w=rgc8005sQAPnjjGt9vluoyq1DPm6ofe-", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Atlanta on the Cheap"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4", "name": "The Best Things to Do in Atlanta in September", "url": "https:\/\/unexpectedatlanta.com\/the-best-things-to-do-in-atlanta-in-september\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-06-19T00:00:00.0000000", "datePublishedDisplayText": "Jun 19, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/unexpectedatlanta.com\/the-best-things-to-do-in-atlanta-in-september", "snippet": "Atlanta Events: September 2024 Experience the Best of Atlanta’s Food &amp; History. Try some of the best eats in Atlanta! The #1-rated Grant Park Past &amp; Future Food Walk is a fun walking tour showcasing the best of Atlanta’s food and history. On this tour, you’ll taste eight modern and Southern foods from three outstanding locally owned food stops.", "dateLastCrawled": "2024-09-17T22:20:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 474, "imageId": "OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+21%2c+2024&d=4781604725337408&mkt=en-US&setlang=en-US&w=L1hd-5qzDrt-bHwGvtgeUoQiW0MCD0-O", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Unexpected Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.5", "name": "20 Spectacular Things To Do This September In Atlanta - Secret Atlanta", "url": "https:\/\/secretatlanta.co\/things-to-do-september\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-03T00:00:00.0000000", "datePublishedDisplayText": "Sep 3, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/secretatlanta.co\/things-to-do-september", "snippet": "When: Saturday, September 7th from 12pm – 10pm. Thousands of people are expected to head to Morris Brown College in Atlanta this September for the legendary Seafood &amp; Caribbean Music festival! Enjoy tons of live music, DJs, and of course, as much mouthwatering seafood and Caribbean dishes as your heart desires! Don’t miss out on all the fun.", "dateLastCrawled": "2024-09-18T21:37:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 316, "imageId": "OIP.mwxBnPRtbggmsDGaNhGACQHaE8"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+21%2c+2024&d=4724958405090999&mkt=en-US&setlang=en-US&w=HkmWmkX0emVkqeBkOazQ7cCeg5xgwMQ3", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Secret Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.6", "name": "Atlanta September 2024: Events, Concerts, Clubs &amp; Things to Do - Experism", "url": "https:\/\/www.experism.com\/atlanta\/september", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.opEpXe5xeSIGAQVvWLFBbAHaEK&w=80&h=80&c=1&pid=5.1", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.experism.com\/atlanta\/september", "snippet": "Sep 12-15, 2024 Historic Fourth Ward Park View on Map. Play Video. The Atlanta Food &amp; Wine Festival highlights the best food, wine, beer &amp; cocktails Atlanta has to offer. This unique event brings in 150+ chefs, sommeliers, mixologists and industry insiders for 5 days of mouthwatering and tempting cuisines &amp; special Tasting Tents.", "dateLastCrawled": "2024-09-20T15:29:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.opEpXe5xeSIGAQVvWLFBbAHaEK&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 266, "imageId": "OIP.opEpXe5xeSIGAQVvWLFBbAHaEK"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+21%2c+2024&d=4642636758720807&mkt=en-US&setlang=en-US&w=PQg3e5KvVPE_Ok9mubm7deAiSBD33s7x", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Experism"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.7", "name": "Explore Atlanta Events for Today, This Week or Weekend - Discover Atlanta", "url": "https:\/\/discoveratlanta.com\/events\/main\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh&w=80&h=80&c=1&pid=5.1", "datePublished": "2010-10-23T00:00:00.0000000", "datePublishedDisplayText": "Oct 23, 2010", "isFamilyFriendly": true, "displayUrl": "https:\/\/discoveratlanta.com\/events\/m", "snippet": "Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... 2024 Metro Atlanta Heart Walk 9\/21\/24. 2024 Million Meal Pack with the Atlanta Hawks 9\/21\/24. BackYARD Fest ... September 21. Saturday. JapanFest Atlanta 9\/21\/24 – 9\/22\/24. What’s Hot . September 26.", "dateLastCrawled": "2024-09-20T21:28:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 289, "imageId": "OIP.FkxSV6JbrWatsFMM3_8dIAHaEh"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+21%2c+2024&d=4819881473017562&mkt=en-US&setlang=en-US&w=jjFkJ7EEAs2P1GMx_bJv9FTM8ziERJGY", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Discover Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.8", "name": "Things To Do in Atlanta This Weekend", "url": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/things-to-do-in-atlanta-this-weekend\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP._omVgwj1ztLZDDfuk8iLpAHaGE&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-16T00:00:00.0000000", "datePublishedFreshnessText": "6 days ago", "isFamilyFriendly": true, "displayUrl": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/things-to-do-in-atlanta-this-weekend", "snippet": "Things To Do in Atlanta This Weekend. By Discover Atlanta Editorial Team. Published September 16, 2024 Last Updated September 20, 2024. Here’s what’s going on in Atlanta this weekend, Sept. 20, 21 and 22. ... and Sept. 21 at the Emma Darnell Aviation Museum. It is the story of Della, who makes cakes, not judgment calls – those she leaves ...", "dateLastCrawled": "2024-09-21T12:13:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP._omVgwj1ztLZDDfuk8iLpAHaGE&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 388, "imageId": "OIP._omVgwj1ztLZDDfuk8iLpAHaGE"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+21%2c+2024&d=5062495582749056&mkt=en-US&setlang=en-US&w=W4oYu9icn6j-KBaHOdxlUp11pCzWRkAm", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Discover Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.9", "name": "Atlanta Festival Calendar: 2024", "url": "https:\/\/unexpectedatlanta.com\/atlanta-festival-calendar-2024\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-05-02T00:00:00.0000000", "datePublishedDisplayText": "May 2, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/unexpectedatlanta.com\/atlanta-festival-calendar-2024", "snippet": "Atlanta Jazz Festival – May 25-27, 2024. Held in Piedmont Park, the Atlanta Jazz Festival is an annual celebration of music, culture, and the art of Jazz. Free to all who join, this Memorial Day Weekend event draws fans, professional musicians, and everyone in between, for a 3-day-long festival showcasing 15+ artists.", "dateLastCrawled": "2024-09-20T03:02:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 474, "imageId": "OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+21%2c+2024&d=4520195830737555&mkt=en-US&setlang=en-US&w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Unexpected Atlanta"}]}, "relatedSearches": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches", "value": [{"text": "atlanta events september 2024", "displayText": "atlanta events september 2024", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+events+september+2024"}, {"text": "things to do in atlanta in september", "displayText": "things to do in atlanta in september", "webSearchUrl": "https:\/\/www.bing.com\/search?q=things+to+do+in+Atlanta+in+september"}, {"text": "festivals in atlanta 2024", "displayText": "festivals in atlanta 2024", "webSearchUrl": "https:\/\/www.bing.com\/search?q=festivals+in+Atlanta+2024"}, {"text": "september events in atlanta", "displayText": "september events in atlanta", "webSearchUrl": "https:\/\/www.bing.com\/search?q=september+events+in+Atlanta"}, {"text": "atlanta in september 2023", "displayText": "atlanta in september 2023", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+in+september+2023"}, {"text": "events in atlanta this weekend", "displayText": "events in atlanta this weekend", "webSearchUrl": "https:\/\/www.bing.com\/search?q=events+in+Atlanta+this+weekend"}, {"text": "atlanta fall festivals 2023", "displayText": "atlanta fall festivals 2023", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+fall+festivals+2023"}, {"text": "atlanta in september", "displayText": "atlanta in september", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+in+september"}]}, "rankingResponse": {"mainline": {"items": [{"answerType": "WebPages", "resultIndex": 0, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0"}}, {"answerType": "WebPages", "resultIndex": 1, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1"}}, {"answerType": "WebPages", "resultIndex": 2, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2"}}, {"answerType": "WebPages", "resultIndex": 3, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3"}}, {"answerType": "WebPages", "resultIndex": 4, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4"}}, {"answerType": "WebPages", "resultIndex": 5, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.5"}}, {"answerType": "WebPages", "resultIndex": 6, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.6"}}, {"answerType": "WebPages", "resultIndex": 7, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.7"}}, {"answerType": "WebPages", "resultIndex": 8, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.8"}}, {"answerType": "WebPages", "resultIndex": 9, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.9"}}, {"answerType": "RelatedSearches", "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches"}}]}}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on September 28, 2024", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"_type": "SearchResponse", "queryContext": {"originalQuery": "things to do in Atlanta on September 28, 2024", "askUserForLocation": true}, "webPages": {"webSearchUrl": "https:\/\/www.bing.com\/search?q=things+to+do+in+Atlanta+on+September+28%2c+2024", "totalEstimatedMatches": 4150000, "value": [{"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0", "name": "Atlanta Events September 2024: Concerts, Shows, Sports...", "url": "https:\/\/atlanta-ga.events\/september\/", "datePublished": "2024-09-19T00:00:00.0000000", "datePublishedFreshnessText": "3 days ago", "isFamilyFriendly": true, "displayUrl": "https:\/\/atlanta-ga.events\/september", "snippet": "Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for September 2024. This guide is your gateway to discovering the best things to do in Atlanta, ... Prices from $28 Avg. price ~ $64. 18 tickets remaining! Tickets; Sep. 21. 2024. 8:00 PM Sat.", "dateLastCrawled": "2024-09-19T19:10:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+28%2c+2024&d=4854189675467386&mkt=en-US&setlang=en-US&w=ShlkdiIwTMefbkmdL1lCvH4mS1bjztOY", "language": "en", "isNavigational": true, "noCache": false, "siteName": "Atlanta Events"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1", "name": "Cool Things To Do in September in Atlanta", "url": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-september-in-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-08-23T00:00:00.0000000", "datePublishedDisplayText": "Aug 23, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-september-in-atlanta", "snippet": "The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta’s most iconic streets, offers fun-filled entertainment, live music — everything from R&amp;B to gospel to alternative favorites — international food court, national artists, celebrities and vendors of all types.", "dateLastCrawled": "2024-09-20T02:07:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 248, "imageId": "OIP.m0ENCio7ytS2o9PQRWDCRgHaD4"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+28%2c+2024&d=4692574351720481&mkt=en-US&setlang=en-US&w=IyKVbCksY9VV74GkIkdoRE4k4vk9MNbJ", "language": "en", "isNavigational": false, "richFacts": [{"label": {"text": "Author"}, "items": [{"text": "Mary Welch"}], "hint": {"text": "BASE:AUTHOR"}}], "noCache": false, "siteName": "Discover Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2", "name": "Atlanta Events - September 2024 - Go South Atlanta", "url": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-september.html", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4&w=80&h=80&c=1&pid=5.1", "isFamilyFriendly": true, "displayUrl": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-september.html", "snippet": "More details. – Dragon Con, August 29 - September 2 2024. One of the biggest annual pop culture conventions in the country. More details. – Atlanta Black Theatre Festival, August 30 - September 1 2024. Annual festival celebrating the Black experience in the performing arts, with plays, panels, masterclasses and other events.", "dateLastCrawled": "2024-09-21T20:12:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 248, "imageId": "OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+28%2c+2024&d=4712253893115949&mkt=en-US&setlang=en-US&w=D6V2DS_RjZRe4oOvzq1dGpeg_GE4sHjt", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Go South Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3", "name": "80+ fall festivals &amp; fairs around metro Atlanta: Fun things to do in ...", "url": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-14T00:00:00.0000000", "datePublishedDisplayText": "Sep 14, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta", "snippet": "♦ Atlanta Streets Alive. September 15, 2024 West End to Grant Park FREE. This is a new route for Fall of 2024! On these Sundays, streets are are temporarily closed to traffic for four hours — so you can come out to walk, run, stroll, bike, scoot, roll, and just have fun.", "dateLastCrawled": "2024-09-21T00:01:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 277, "imageId": "OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+28%2c+2024&d=5012794228343791&mkt=en-US&setlang=en-US&w=rgc8005sQAPnjjGt9vluoyq1DPm6ofe-", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Atlanta on the Cheap"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4", "name": "20 Spectacular Things To Do This September In Atlanta - Secret Atlanta", "url": "https:\/\/secretatlanta.co\/things-to-do-september\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-03T00:00:00.0000000", "datePublishedDisplayText": "Sep 3, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/secretatlanta.co\/things-to-do-september", "snippet": "When: Saturday, September 7th from 12pm – 10pm. Thousands of people are expected to head to Morris Brown College in Atlanta this September for the legendary Seafood &amp; Caribbean Music festival! Enjoy tons of live music, DJs, and of course, as much mouthwatering seafood and Caribbean dishes as your heart desires! Don’t miss out on all the fun.", "dateLastCrawled": "2024-09-18T21:37:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 316, "imageId": "OIP.mwxBnPRtbggmsDGaNhGACQHaE8"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+28%2c+2024&d=4724958405090999&mkt=en-US&setlang=en-US&w=HkmWmkX0emVkqeBkOazQ7cCeg5xgwMQ3", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Secret Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.5", "name": "The Best Things to Do in Atlanta in September", "url": "https:\/\/unexpectedatlanta.com\/the-best-things-to-do-in-atlanta-in-september\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-06-19T00:00:00.0000000", "datePublishedDisplayText": "Jun 19, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/unexpectedatlanta.com\/the-best-things-to-do-in-atlanta-in-september", "snippet": "Atlanta Events: September 2024 Experience the Best of Atlanta’s Food &amp; History. Try some of the best eats in Atlanta! The #1-rated Grant Park Past &amp; Future Food Walk is a fun walking tour showcasing the best of Atlanta’s food and history. On this tour, you’ll taste eight modern and Southern foods from three outstanding locally owned food stops.", "dateLastCrawled": "2024-09-17T22:20:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 474, "imageId": "OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+28%2c+2024&d=4781604725337408&mkt=en-US&setlang=en-US&w=L1hd-5qzDrt-bHwGvtgeUoQiW0MCD0-O", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Unexpected Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.6", "name": "Look Ahead: Local events and things to do this week, September 22-28", "url": "https:\/\/www.sltrib.com\/artsliving\/2024\/09\/22\/look-ahead-local-events-things-do\/", "datePublished": "2024-09-22T00:00:00.0000000", "datePublishedFreshnessText": "16 hours ago", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.sltrib.com\/artsliving\/2024\/09\/22\/look-ahead-local-events-things-do", "snippet": "Sept. 24. The end of Red Butte’s summer. The final concert in Red Butte Garden’s summer concert series happens Tuesday, with the acoustic guitar duo Rodrigo y Gabriela.Opening the show is Okan ...", "dateLastCrawled": "2024-09-22T13:54:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+28%2c+2024&d=593154759898&mkt=en-US&setlang=en-US&w=MzxxUrr8I5BY3fYcumvveEXlMkuDWsRV", "language": "en", "isNavigational": false, "noCache": false, "siteName": "The Salt Lake Tribune"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.7", "name": "Atlanta September 2024: Events, Concerts, Clubs &amp; Things to Do - Experism", "url": "https:\/\/www.experism.com\/atlanta\/september", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.opEpXe5xeSIGAQVvWLFBbAHaEK&w=80&h=80&c=1&pid=5.1", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.experism.com\/atlanta\/september", "snippet": "Sep 12-15, 2024 Historic Fourth Ward Park View on Map. Play Video. The Atlanta Food &amp; Wine Festival highlights the best food, wine, beer &amp; cocktails Atlanta has to offer. This unique event brings in 150+ chefs, sommeliers, mixologists and industry insiders for 5 days of mouthwatering and tempting cuisines &amp; special Tasting Tents.", "dateLastCrawled": "2024-09-20T15:29:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.opEpXe5xeSIGAQVvWLFBbAHaEK&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 266, "imageId": "OIP.opEpXe5xeSIGAQVvWLFBbAHaEK"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+28%2c+2024&d=4642636758720807&mkt=en-US&setlang=en-US&w=PQg3e5KvVPE_Ok9mubm7deAiSBD33s7x", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Experism"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.8", "name": "Things to Do in Atlanta in 2024: Events, festivals, and exhibits ...", "url": "https:\/\/www.thrillist.com\/events\/atlanta\/events-calendar-in-atlanta-this-year", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-07-09T00:00:00.0000000", "datePublishedDisplayText": "Jul 9, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.thrillist.com\/events\/atlanta\/events-calendar-in-atlanta-this-year", "snippet": "23 Things You Have to Do in Atlanta This Year, From One Musicfest to Homecoming ... September 28–29, 2024 Old Fourth Ward One of the city’s most anticipated events is back in September. The ...", "dateLastCrawled": "2024-09-20T01:10:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 316, "imageId": "OIP.WUHCPl7_RpepE3pSEdAEMgHaE8"}, "language": "en", "isNavigational": false, "noCache": true, "siteName": "Thrillist"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.9", "name": "Explore Atlanta Events for Today, This Week or Weekend - Discover Atlanta", "url": "https:\/\/discoveratlanta.com\/events\/main\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh&w=80&h=80&c=1&pid=5.1", "datePublished": "2010-10-23T00:00:00.0000000", "datePublishedDisplayText": "Oct 23, 2010", "isFamilyFriendly": true, "displayUrl": "https:\/\/discoveratlanta.com\/events\/m", "snippet": "Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Every Saturday until 09\/28\/2024. Protect Yo Energy Con 9\/20\/24 – 11\/9\/24. ... September 28. Saturday. Sweet Auburn Music Fest 9\/28\/24 – 9\/29\/24. Editors’ Pick . October 01.", "dateLastCrawled": "2024-09-20T21:28:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 289, "imageId": "OIP.FkxSV6JbrWatsFMM3_8dIAHaEh"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+September+28%2c+2024&d=4819881473017562&mkt=en-US&setlang=en-US&w=jjFkJ7EEAs2P1GMx_bJv9FTM8ziERJGY", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Discover Atlanta"}], "someResultsRemoved": true}, "relatedSearches": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches", "value": [{"text": "atlanta events september 2024", "displayText": "atlanta events september 2024", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+events+september+2024"}, {"text": "things to do in atlanta in september", "displayText": "things to do in atlanta in september", "webSearchUrl": "https:\/\/www.bing.com\/search?q=things+to+do+in+Atlanta+in+september"}, {"text": "september events in atlanta", "displayText": "september events in atlanta", "webSearchUrl": "https:\/\/www.bing.com\/search?q=september+events+in+Atlanta"}, {"text": "festivals in atlanta 2024", "displayText": "festivals in atlanta 2024", "webSearchUrl": "https:\/\/www.bing.com\/search?q=festivals+in+Atlanta+2024"}, {"text": "events in atlanta this weekend", "displayText": "events in atlanta this weekend", "webSearchUrl": "https:\/\/www.bing.com\/search?q=events+in+Atlanta+this+weekend"}, {"text": "atlanta in september", "displayText": "atlanta in september", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+in+september"}, {"text": "atlanta in september 2023", "displayText": "atlanta in september 2023", "webSearchUrl": "https:\/\/www.bing.com\/search?q=Atlanta+in+september+2023"}, {"text": "underground atlanta september events", "displayText": "underground atlanta september events", "webSearchUrl": "https:\/\/www.bing.com\/search?q=underground+Atlanta+september+events"}]}, "rankingResponse": {"mainline": {"items": [{"answerType": "WebPages", "resultIndex": 0, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0"}}, {"answerType": "WebPages", "resultIndex": 1, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1"}}, {"answerType": "WebPages", "resultIndex": 2, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2"}}, {"answerType": "WebPages", "resultIndex": 3, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3"}}, {"answerType": "WebPages", "resultIndex": 4, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4"}}, {"answerType": "WebPages", "resultIndex": 5, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.5"}}, {"answerType": "WebPages", "resultIndex": 6, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.6"}}, {"answerType": "WebPages", "resultIndex": 7, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.7"}}, {"answerType": "WebPages", "resultIndex": 8, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.8"}}, {"answerType": "WebPages", "resultIndex": 9, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.9"}}, {"answerType": "RelatedSearches", "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches"}}]}}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on October 5, 2024", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"_type": "SearchResponse", "queryContext": {"originalQuery": "things to do in Atlanta on October 5, 2024", "askUserForLocation": true}, "webPages": {"webSearchUrl": "https:\/\/www.bing.com\/search?q=things+to+do+in+Atlanta+on+October+5%2c+2024", "totalEstimatedMatches": 6350000, "value": [{"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0", "name": "Atlanta Events - October 2024 - Go South Atlanta", "url": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-october.html", "isFamilyFriendly": true, "displayUrl": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-october.html", "snippet": "Wrestling event at State Farm Arena. More details. – Festival on Ponce, October 5-6 2024. Twice-annual arts festival in Olmsted Linear Park, with 125+ vendors of fine art, folk art and outsider art, plus food and drink, and children’s activities. More details. – Candler Park Fall Fest, October 5-6 2024.", "dateLastCrawled": "2024-09-21T01:19:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+5%2c+2024&d=4907344187173452&mkt=en-US&setlang=en-US&w=B_sqEJ0YsMi3mkDLUvcm_m1afK8iBTpt", "language": "en", "isNavigational": true, "noCache": false, "siteName": "Go South Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1", "name": "Cool Things To Do in October in Atlanta", "url": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-october-in-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-13T00:00:00.0000000", "datePublishedDisplayText": "Sep 13, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-october-in-atlanta", "snippet": "Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3. Read More.", "dateLastCrawled": "2024-09-22T04:12:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 248, "imageId": "OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+5%2c+2024&d=4714289705203876&mkt=en-US&setlang=en-US&w=ZgosuhXzNBJfz-dx9C91WDDECk49pD7V", "language": "en", "isNavigational": false, "richFacts": [{"label": {"text": "Author"}, "items": [{"text": "Mary Welch"}], "hint": {"text": "BASE:AUTHOR"}}], "noCache": false, "siteName": "Discover Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2", "name": "Atlanta Events October 2024: Concerts, Shows, Sports...", "url": "https:\/\/atlanta-ga.events\/october\/", "isFamilyFriendly": true, "displayUrl": "https:\/\/atlanta-ga.events\/october", "snippet": "Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You’ll find a detailed list of all the most engaging and exciting activities happening throughout the month.", "dateLastCrawled": "2024-09-22T04:13:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+5%2c+2024&d=4898908879324774&mkt=en-US&setlang=en-US&w=qXhnhPcHzBqzxHogtJy0oBuKycEPAm_-", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Atlanta Events"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3", "name": "80+ fall festivals &amp; fairs around metro Atlanta: Fun things to do in ...", "url": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-14T00:00:00.0000000", "datePublishedDisplayText": "Sep 14, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta", "snippet": "October 5, 2024 Town Center Park Suwanee FREE admission. This all-day music festival takes place in downtown Suwanee. Admission, parking, and entertainment is free. The BBQ tasting tickets s are $1 each, and each taste requires one to four tickets. You can also buy a full plated barbecue meal. There is also a kids fun area and an artist market.", "dateLastCrawled": "2024-09-21T00:01:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 277, "imageId": "OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+5%2c+2024&d=5012794228343791&mkt=en-US&setlang=en-US&w=rgc8005sQAPnjjGt9vluoyq1DPm6ofe-", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Atlanta on the Cheap"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4", "name": "Atlanta in October | Concerts, Festivals, Tickets &amp; Tour Dates - Songkick", "url": "https:\/\/www.songkick.com\/metro-areas\/4120-us-atlanta\/october-2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.songkick.com\/metro-areas\/4120-us-atlanta\/october-2024", "snippet": "Concerts in Atlanta in October. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 544 upcoming events. Filter by artist. ... Sat 12 Oct 2024 Atlanta Coliseum Duluth, GA, US. Kompany. Sat 02 Nov 2024 Believe Music Hall Atlanta, GA, US.", "dateLastCrawled": "2024-09-17T13:20:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+5%2c+2024&d=4726551829817777&mkt=en-US&setlang=en-US&w=GyNsJVWjOONlYyuTqLMEVsVddRXByGfo", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Songkick"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.5", "name": "26 Thrilling Things To Do In Atlanta This October - Secret Atlanta", "url": "https:\/\/secretatlanta.co\/things-to-do-october\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.oVWYpGkEV4zoUw2uzcrcYgHaD6&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-05T00:00:00.0000000", "datePublishedDisplayText": "Sep 5, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/secretatlanta.co\/things-to-do-october", "snippet": "Without further ado, here are 26 incredible things to do in Atlanta this October! 1. ... September 20, 2024. 18 Frightfully Fun Halloween Events Happening In Atlanta. September 20, 2024. This Creepy Corn Maze In Buford Comes Alive With Spooks &amp; Haunts At Sunset.", "dateLastCrawled": "2024-09-20T22:15:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.oVWYpGkEV4zoUw2uzcrcYgHaD6&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 250, "imageId": "OIP.oVWYpGkEV4zoUw2uzcrcYgHaD6"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+5%2c+2024&d=4673096675364470&mkt=en-US&setlang=en-US&w=ixV0zLI6M09NFCQACSuK0PesDCkO7vK5", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Secret Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.6", "name": "Atlanta Festival Calendar: 2024", "url": "https:\/\/unexpectedatlanta.com\/atlanta-festival-calendar-2024\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-05-02T00:00:00.0000000", "datePublishedDisplayText": "May 2, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/unexpectedatlanta.com\/atlanta-festival-calendar-2024", "snippet": "Atlanta Jazz Festival – May 25-27, 2024. Held in Piedmont Park, the Atlanta Jazz Festival is an annual celebration of music, culture, and the art of Jazz. Free to all who join, this Memorial Day Weekend event draws fans, professional musicians, and everyone in between, for a 3-day-long festival showcasing 15+ artists.", "dateLastCrawled": "2024-09-20T03:02:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 474, "imageId": "OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+5%2c+2024&d=4520195830737555&mkt=en-US&setlang=en-US&w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Unexpected Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.7", "name": "The Best Things to Do in Atlanta in October", "url": "https:\/\/unexpectedatlanta.com\/the-best-things-to-do-in-atlanta-in-october\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.lEMypzNNWpS-6tdPg0Hm1gHaHa&w=80&h=80&c=1&pid=5.1", "datePublished": "2023-09-25T00:00:00.0000000", "datePublishedDisplayText": "Sep 25, 2023", "isFamilyFriendly": true, "displayUrl": "https:\/\/unexpectedatlanta.com\/the-best-things-to-do-in-atlanta-in-october", "snippet": "Catch Ilana Glazer at the Tabernacle on October 23, 2023. Best known for her role as Ilana Wexler in Broad City, Glazer’s live comedy has been called bold, delving into modern social and political issues with unapologetic, empowering, and satirical takes. Plus, catch Nick Offerman at Atlanta Symphony Hall.", "dateLastCrawled": "2024-09-17T11:39:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.lEMypzNNWpS-6tdPg0Hm1gHaHa&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 474, "imageId": "OIP.lEMypzNNWpS-6tdPg0Hm1gHaHa"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+5%2c+2024&d=4733754493195045&mkt=en-US&setlang=en-US&w=wJ10aadQT5xoqbb3_3asNbrAJ_n6Hq4i", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Unexpected Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.8", "name": "Things to Do in Atlanta in 2024: Events, festivals, and exhibits ...", "url": "https:\/\/www.thrillist.com\/events\/atlanta\/events-calendar-in-atlanta-this-year", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-07-09T00:00:00.0000000", "datePublishedDisplayText": "Jul 9, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.thrillist.com\/events\/atlanta\/events-calendar-in-atlanta-this-year", "snippet": "August 23–25, 2024. The Fox Theatre. Every August, the Fox Theatre pays homage to its history as an iconic movie venue with its annual Coca Cola Summer Film Festival. This year’s edition ...", "dateLastCrawled": "2024-09-20T01:10:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 316, "imageId": "OIP.WUHCPl7_RpepE3pSEdAEMgHaE8"}, "language": "en", "isNavigational": false, "noCache": true, "siteName": "Thrillist"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.9", "name": "Concerts In Atlanta, October 2024 - Go South Atlanta", "url": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-concerts-october.html", "isFamilyFriendly": true, "displayUrl": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-concerts-october.html", "snippet": "Concerts On Friday October 11 2024. – Lake Street Dive at The Eastern. More details. – Aerosmith: Peace Out The Farewell Tour at State Farm Arena. More details. – The Black Jacket Symphony: Performing Led Zeppelin’s ‘IV’. Tribute concert at Atlanta Symphony Hall. More details. – Loveless at The Masquerade.", "dateLastCrawled": "2024-09-20T17:04:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+5%2c+2024&d=4991061691676766&mkt=en-US&setlang=en-US&w=BuowMU6wITTdrEf4HxlkWHNc_Lnct5jK", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Go South Atlanta"}]}, "relatedSearches": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches", "value": [{"text": "october 2024 atlanta events", "displayText": "october 2024 atlanta events", "webSearchUrl": "https:\/\/www.bing.com\/search?q=october+2024+atlanta+events"}, {"text": "things to do in october atlanta", "displayText": "things to do in october atlanta", "webSearchUrl": "https:\/\/www.bing.com\/search?q=things+to+do+in+october+atlanta"}, {"text": "festivals in atlanta 2024", "displayText": "festivals in atlanta 2024", "webSearchUrl": "https:\/\/www.bing.com\/search?q=festivals+in+atlanta+2024"}, {"text": "atlanta 2024 events", "displayText": "atlanta 2024 events", "webSearchUrl": "https:\/\/www.bing.com\/search?q=atlanta+2024+events"}, {"text": "atlanta fall festivals 2023", "displayText": "atlanta fall festivals 2023", "webSearchUrl": "https:\/\/www.bing.com\/search?q=atlanta+fall+festivals+2023"}, {"text": "fall festivals near atlanta ga", "displayText": "fall festivals near atlanta ga", "webSearchUrl": "https:\/\/www.bing.com\/search?q=fall+festivals+near+atlanta+ga"}, {"text": "festivals in atlanta in september", "displayText": "festivals in atlanta in september", "webSearchUrl": "https:\/\/www.bing.com\/search?q=festivals+in+atlanta+in+september"}, {"text": "cool things to do in october", "displayText": "cool things to do in october", "webSearchUrl": "https:\/\/www.bing.com\/search?q=cool+things+to+do+in+october"}]}, "rankingResponse": {"mainline": {"items": [{"answerType": "WebPages", "resultIndex": 0, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0"}}, {"answerType": "WebPages", "resultIndex": 1, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1"}}, {"answerType": "WebPages", "resultIndex": 2, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2"}}, {"answerType": "WebPages", "resultIndex": 3, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3"}}, {"answerType": "WebPages", "resultIndex": 4, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4"}}, {"answerType": "WebPages", "resultIndex": 5, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.5"}}, {"answerType": "WebPages", "resultIndex": 6, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.6"}}, {"answerType": "WebPages", "resultIndex": 7, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.7"}}, {"answerType": "WebPages", "resultIndex": 8, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.8"}}, {"answerType": "WebPages", "resultIndex": 9, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.9"}}, {"answerType": "RelatedSearches", "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches"}}]}}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on October 12, 2024", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"_type": "SearchResponse", "queryContext": {"originalQuery": "things to do in Atlanta on October 12, 2024", "askUserForLocation": true}, "webPages": {"webSearchUrl": "https:\/\/www.bing.com\/search?q=things+to+do+in+Atlanta+on+October+12%2c+2024", "totalEstimatedMatches": 4920000, "value": [{"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0", "name": "Atlanta Events - October 2024 | Go South Atlanta", "url": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-october.html", "isFamilyFriendly": true, "displayUrl": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-in-october.html", "snippet": "Wrestling event at State Farm Arena. More details. – Festival on Ponce, October 5-6 2024. Twice-annual arts festival in Olmsted Linear Park, with 125+ vendors of fine art, folk art and outsider art, plus food and drink, and children’s activities. More details. – Candler Park Fall Fest, October 5-6 2024.", "dateLastCrawled": "2024-09-21T01:19:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+12%2c+2024&d=4907344187173452&mkt=en-US&setlang=en-US&w=B_sqEJ0YsMi3mkDLUvcm_m1afK8iBTpt", "language": "en", "isNavigational": true, "noCache": false, "siteName": "Go South Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1", "name": "Atlanta Events October 2024: Concerts, Shows, Sports...", "url": "https:\/\/atlanta-ga.events\/october\/", "isFamilyFriendly": true, "displayUrl": "https:\/\/atlanta-ga.events\/october", "snippet": "Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You’ll find a detailed list of all the most engaging and exciting activities happening throughout the month.", "dateLastCrawled": "2024-09-22T04:13:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+12%2c+2024&d=4898908879324774&mkt=en-US&setlang=en-US&w=qXhnhPcHzBqzxHogtJy0oBuKycEPAm_-", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Atlanta Events"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2", "name": "Cool Things To Do in October in Atlanta", "url": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-october-in-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-13T00:00:00.0000000", "datePublishedDisplayText": "Sep 13, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/discoveratlanta.com\/stories\/things-to-do\/cool-things-to-do-in-october-in-atlanta", "snippet": "Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3. Read More.", "dateLastCrawled": "2024-09-20T08:12:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 248, "imageId": "OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+12%2c+2024&d=4714289705203876&mkt=en-US&setlang=en-US&w=ZgosuhXzNBJfz-dx9C91WDDECk49pD7V", "language": "en", "isNavigational": false, "richFacts": [{"label": {"text": "Author"}, "items": [{"text": "Mary Welch"}], "hint": {"text": "BASE:AUTHOR"}}], "noCache": false, "siteName": "Discover Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3", "name": "80+ fall festivals &amp; fairs around metro Atlanta: Fun things to do in ...", "url": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-09-14T00:00:00.0000000", "datePublishedDisplayText": "Sep 14, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.atlantaonthecheap.com\/fall-festivals-atlanta", "snippet": "October 12 and 13, 2024 October 19 and 20, 2024 Ellijay, GA . This is one of the few festivals outside metro Atlanta we include. It takes place in North Georgia, over two weekends in October. Spend a day on the farm, pick your own apples, visit over 100 vendor booths for arts, crafts, and apple-based food items. Well worth the drive!", "dateLastCrawled": "2024-09-21T00:01:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 277, "imageId": "OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+12%2c+2024&d=5012794228343791&mkt=en-US&setlang=en-US&w=rgc8005sQAPnjjGt9vluoyq1DPm6ofe-", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Atlanta on the Cheap"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4", "name": "Atlanta in October | Concerts, Festivals, Tickets &amp; Tour Dates | Songkick", "url": "https:\/\/www.songkick.com\/metro-areas\/4120-us-atlanta\/october-2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.songkick.com\/metro-areas\/4120-us-atlanta\/october-2024", "snippet": "Sat 12 Oct 2024 Atlanta Coliseum Duluth, GA, US. Kompany. Sat 02 Nov 2024 Believe Music Hall Atlanta, GA, US. Mickey Guyton. Sat 28 Sep 2024 Eddie&#39;s Attic Decatur, GA, US. Herb Alpert. Fri 13 Dec 2024 Buckhead Theatre Atlanta, GA, US. Monsoon. Sat 12 Oct 2024 ...", "dateLastCrawled": "2024-09-17T13:20:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+12%2c+2024&d=4726551829817777&mkt=en-US&setlang=en-US&w=GyNsJVWjOONlYyuTqLMEVsVddRXByGfo", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Songkick"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.5", "name": "Atlanta Festival Calendar: 2024", "url": "https:\/\/unexpectedatlanta.com\/atlanta-festival-calendar-2024\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-05-02T00:00:00.0000000", "datePublishedDisplayText": "May 2, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/unexpectedatlanta.com\/atlanta-festival-calendar-2024", "snippet": "This experience was named by Conde Nast as one of the best things to do in Atlanta! February Festivals in Atlanta Atlanta Winter Beer Festival – February 10, 2024 ... October 12-13, 2024. The Atlanta Pride Festival and Parade is a vibrant event that celebrates and supports the LGBTQ+ community. By attending, be a part of a joyful and ...", "dateLastCrawled": "2024-09-20T03:02:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 474, "imageId": "OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+12%2c+2024&d=4520195830737555&mkt=en-US&setlang=en-US&w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Unexpected Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.6", "name": "Atlanta October 2024: Events, Concerts, Clubs &amp; Things to Do | Experism", "url": "https:\/\/www.experism.com\/atlanta\/october", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.8x_jPSRdmgimGRTNJq3pngAAAA&w=80&h=80&c=1&pid=5.1", "isFamilyFriendly": true, "displayUrl": "https:\/\/www.experism.com\/atlanta\/october", "snippet": "Atlanta Horror Film Festival. Oct 4-6, 2024 Limelight Theater View on Map. You are cordially invited to the 18th Annual Atlanta Horror Film Festival held at Limelight Theater from October 4 - 6, 2024. Don&#39;t miss this spooktacular collection of the best indie horror films from across the globe, including world premieres, short films, animation ...", "dateLastCrawled": "2024-09-17T21:54:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.8x_jPSRdmgimGRTNJq3pngAAAA&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 316, "imageId": "OIP.8x_jPSRdmgimGRTNJq3pngAAAA"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+12%2c+2024&d=5050860516094956&mkt=en-US&setlang=en-US&w=OllQT5kZyN343ZtIpGRATFj2b4uP5Oth", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Experism"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.7", "name": "24 Exciting Things To Add To Your Atlanta Bucket List | Secret Atlanta", "url": "https:\/\/secretatlanta.co\/2024-atlanta-bucket-list\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.Jx474uFzP4iYwa3hHq7X7AHaFn&w=80&h=80&c=1&pid=5.1", "datePublished": "2024-01-25T00:00:00.0000000", "datePublishedDisplayText": "Jan 25, 2024", "isFamilyFriendly": true, "displayUrl": "https:\/\/secretatlanta.co\/2024-atlanta-bucket-list", "snippet": "Here are 24 things to add to your Atlanta bucket list! 1. ... October 4, 2024 7:00 PM (+ more dates) The Chapel on Sycamore. ... September 12, 2024. Spooky Scarecrows Take Over The Atlanta Botanical Garden For Fall. September 12, 2024. SHARE THE ARTICLE. Tags: candlelight concert.", "dateLastCrawled": "2024-09-14T02:13:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.Jx474uFzP4iYwa3hHq7X7AHaFn&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 359, "imageId": "OIP.Jx474uFzP4iYwa3hHq7X7AHaFn"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+12%2c+2024&d=4710587443139341&mkt=en-US&setlang=en-US&w=2gj7DwR9-YxeIPl9aAYvzasJhzxpOrCq", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Secret Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.8", "name": "The Best Things to Do in Atlanta in October", "url": "https:\/\/unexpectedatlanta.com\/the-best-things-to-do-in-atlanta-in-october\/", "thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.lEMypzNNWpS-6tdPg0Hm1gHaHa&w=80&h=80&c=1&pid=5.1", "datePublished": "2023-09-25T00:00:00.0000000", "datePublishedDisplayText": "Sep 25, 2023", "isFamilyFriendly": true, "displayUrl": "https:\/\/unexpectedatlanta.com\/the-best-things-to-do-in-atlanta-in-october", "snippet": "Catch Ilana Glazer at the Tabernacle on October 23, 2023. Best known for her role as Ilana Wexler in Broad City, Glazer’s live comedy has been called bold, delving into modern social and political issues with unapologetic, empowering, and satirical takes. Plus, catch Nick Offerman at Atlanta Symphony Hall.", "dateLastCrawled": "2024-09-17T11:39:00.0000000Z", "primaryImageOfPage": {"thumbnailUrl": "https:\/\/www.bing.com\/th?id=OIP.lEMypzNNWpS-6tdPg0Hm1gHaHa&w=80&h=80&c=1&pid=5.1", "width": 80, "height": 80, "sourceWidth": 474, "sourceHeight": 474, "imageId": "OIP.lEMypzNNWpS-6tdPg0Hm1gHaHa"}, "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+12%2c+2024&d=4733754493195045&mkt=en-US&setlang=en-US&w=wJ10aadQT5xoqbb3_3asNbrAJ_n6Hq4i", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Unexpected Atlanta"}, {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.9", "name": "Concerts In Atlanta, October 2024 | Go South Atlanta", "url": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-concerts-october.html", "isFamilyFriendly": true, "displayUrl": "https:\/\/gosouthatlanta.com\/things-to-do\/events\/atlanta-concerts-october.html", "snippet": "Concerts On Friday October 11 2024. – Lake Street Dive at The Eastern. More details. – Aerosmith: Peace Out The Farewell Tour at State Farm Arena. More details. – The Black Jacket Symphony: Performing Led Zeppelin’s ‘IV’. Tribute concert at Atlanta Symphony Hall. More details. – Loveless at The Masquerade.", "dateLastCrawled": "2024-09-20T17:04:00.0000000Z", "cachedPageUrl": "http:\/\/cc.bingj.com\/cache.aspx?q=things+to+do+in+Atlanta+on+October+12%2c+2024&d=4991061691676766&mkt=en-US&setlang=en-US&w=BuowMU6wITTdrEf4HxlkWHNc_Lnct5jK", "language": "en", "isNavigational": false, "noCache": false, "siteName": "Go South Atlanta"}]}, "images": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#Images", "readLink": "https:\/\/api.bing.microsoft.com\/api\/v7\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&qpvt=things+to+do+in+Atlanta+on+October+12%2c+2024", "webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&qpvt=things+to+do+in+Atlanta+on+October+12%2c+2024", "isFamilyFriendly": true, "value": [{"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=7149423C541A46F11DBC041B08DF472012E1EBD1&FORM=IQFRBA", "name": "7 Best Things to Do in Atlanta, GA", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1&pid=Api", "datePublished": "2022-11-30T00:54:00.0000000Z", "contentUrl": "https:\/\/uphomes.com\/storage\/blogposts\/February2023\/medium\/RKuDJcqErH176HD3LyWV.png", "hostPageUrl": "https:\/\/uphomes.com\/blog\/things-to-do-atlanta-ga\/", "contentSize": "1985403 B", "encodingFormat": "png", "hostPageDisplayUrl": "https:\/\/uphomes.com\/blog\/things-to-do-atlanta-ga\/", "width": 1096, "height": 1012, "thumbnail": {"width": 474, "height": 437}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=6AF242267488E704AF02F9CB6D6B85FECFA1AF2E&FORM=IQFRBA", "name": "35+ Best Things to Do in Atlanta in 2024 - Unexpected Atlanta", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.HwzfdFE6-tSHfhoFlTxXKQHaHa&pid=Api", "datePublished": "2021-05-28T10:54:00.0000000Z", "contentUrl": "https:\/\/unexpectedatlanta.com\/wp-content\/uploads\/2022\/12\/Best-Things-to-Do-in-Atlanta-in-2023.png", "hostPageUrl": "https:\/\/unexpectedatlanta.com\/best-things-to-do-in-atlanta\/", "contentSize": "43967 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/unexpectedatlanta.com\/best-things-to-do-in-atlanta\/", "width": 600, "height": 600, "thumbnail": {"width": 474, "height": 474}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=41B8CEB6C0CE4ADA8F198155C3CED86FCA48F23B&FORM=IQFRBA", "name": "27 UNIQUE Things to Do in Atlanta [in 2024]", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.fLae3sxHYp3BVcZ5UZwd8gHaLM&pid=Api", "datePublished": "2019-10-13T09:41:00.0000000Z", "contentUrl": "https:\/\/www.thebrokebackpacker.com\/wp-content\/uploads\/2019\/10\/things-to-do-atlanta-pin-1.jpg", "hostPageUrl": "https:\/\/www.thebrokebackpacker.com\/things-to-do-in-atlanta-georgia\/", "contentSize": "49017 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/www.thebrokebackpacker.com\/things-to-do-in-atlanta-georgia\/", "width": 512, "height": 774, "thumbnail": {"width": 474, "height": 716}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=CEBCC48B5F475941881645BBB33DD1273AFA3BD3&FORM=IQFRBA", "name": "The Best Things to Do in Atlanta in October - Unexpected Atlanta", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.lEMypzNNWpS-6tdPg0Hm1gHaHa&pid=Api", "datePublished": "2023-09-25T16:33:00.0000000Z", "contentUrl": "https:\/\/unexpectedatlanta.com\/wp-content\/uploads\/2023\/09\/The-Best-Things-to-Do-in-Atlanta-in-October.png", "hostPageUrl": "https:\/\/unexpectedatlanta.com\/the-best-things-to-do-in-atlanta-in-october\/", "contentSize": "249284 B", "encodingFormat": "png", "hostPageDisplayUrl": "https:\/\/unexpectedatlanta.com\/the-best-things-to-do-in-atlanta-in-october\/", "width": 600, "height": 600, "thumbnail": {"width": 474, "height": 474}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=1CD06FECC1BFFFC536537A6ACE80AE2F0738C85E&FORM=IQFRBA", "name": "20 Free Things To Do In Atlanta, Georgia | Atlanta travel tips ...", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.wxOD0OTZ89VvVuQNnfz51wHaL0&pid=Api", "datePublished": "2022-11-25T02:13:00.0000000Z", "contentUrl": "https:\/\/i.pinimg.com\/originals\/7d\/b9\/cb\/7db9cb75f1804a65a7512408c3763aea.jpg", "hostPageUrl": "https:\/\/www.pinterest.com\/pin\/42432421477082343\/", "contentSize": "127267 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/www.pinterest.com\/pin\/42432421477082343\/", "width": 736, "height": 1174, "thumbnail": {"width": 474, "height": 756}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=6992E50E8C164A2F38D79E7A0589A9E505A49484&FORM=IQFRBA", "name": "12 Things to do in Atlanta, GA With Kids [2024 LIST] | Atlanta Family Fun", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.Agse3pS4C61ss3RxvVgySwHaEK&pid=Api", "datePublished": "2020-04-08T11:34:00.0000000Z", "contentUrl": "https:\/\/wirksmoving.com\/wp-content\/uploads\/2020\/04\/Things-to-do-in-Atlanta-with-Kids-1536x864.png", "hostPageUrl": "https:\/\/wirksmoving.com\/blog\/things-to-do-in-atlanta-ga-with-kids\/", "contentSize": "419299 B", "encodingFormat": "png", "hostPageDisplayUrl": "https:\/\/wirksmoving.com\/blog\/things-to-do-in-atlanta-ga-with-kids\/", "width": 1536, "height": 864, "thumbnail": {"width": 474, "height": 266}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=0FB889EE6E37AC260B6DFC60CE211E7CD723568B&FORM=IQFRBA", "name": "50 Fun Things to Do in Atlanta, Georgia (Updated 2024)", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.Yac324FtG-dH_KY8XaUTZQHaME&pid=Api", "datePublished": "2017-06-06T12:00:00.0000000Z", "contentUrl": "https:\/\/www.frostandsun.com\/wp-content\/uploads\/2017\/11\/things-to-do-in-atlanta-pin-me-1-628x1024.jpg.webp", "hostPageUrl": "https:\/\/www.frostandsun.com\/things-to-do-in-atlanta\/", "contentSize": "59730 B", "encodingFormat": "unknown", "hostPageDisplayUrl": "https:\/\/www.frostandsun.com\/things-to-do-in-atlanta\/", "width": 628, "height": 1024, "thumbnail": {"width": 474, "height": 772}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=08DAD9253511A657404BCC3EFE3705B0FF14F586&FORM=IQFRBA", "name": "Top 10 Things to Do in Atlanta 2024 - WOW Travel", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.gpxEMgIaDnDSvyMtPJjjdgHaE6&pid=Api", "datePublished": "2022-06-27T11:35:00.0000000Z", "contentUrl": "https:\/\/wowtravel.me\/wp-content\/uploads\/2022\/06\/Top-10-Things-to-Do-in-Atlanta.jpg", "hostPageUrl": "https:\/\/wowtravel.me\/things-to-do-in-atlanta-this-week\/", "contentSize": "82711 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/wowtravel.me\/things-to-do-in-atlanta-this-week\/", "width": 1024, "height": 680, "thumbnail": {"width": 474, "height": 314}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=02FE7611C5C1EB1A25FB01C84B9F2E6B94434679&FORM=IQFRBA", "name": "The 24 Best Things to Do in Atlanta in 2024 | Our Top Picks", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.M9nQL4kFM4rwNQiRkhAekAHaEo&pid=Api", "datePublished": "2022-05-03T19:37:00.0000000Z", "contentUrl": "https:\/\/travellersworldwide.com\/wp-content\/uploads\/2022\/05\/shutterstock_1039281055.jpg", "hostPageUrl": "https:\/\/travellersworldwide.com\/best-things-to-do-in-atlanta\/", "contentSize": "1912569 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/travellersworldwide.com\/best-things-to-do-in-atlanta\/", "width": 1600, "height": 1000, "thumbnail": {"width": 474, "height": 296}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=85EEFD5A63E8420E6067DDF98A07B2F0FE6BEEC0&FORM=IQFRBA", "name": "Top 10 Things To Do in Atlanta Georgia | A Local&#39;s Atlanta Travel Guide ...", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.Twqjja_QS8r-DEoNYr3kzAHaEK&pid=Api", "datePublished": "2021-01-30T23:01:00.0000000Z", "contentUrl": "https:\/\/i.ytimg.com\/vi\/Vl6GPUuj1mc\/maxresdefault.jpg", "hostPageUrl": "https:\/\/www.youtube.com\/watch?v=Vl6GPUuj1mc", "contentSize": "179056 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/www.youtube.com\/watch?v=Vl6GPUuj1mc", "width": 1280, "height": 720, "thumbnail": {"width": 474, "height": 266}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=EED9BCE9A4C18742AACBFE3295728763F0253381&FORM=IQFRBA", "name": "101 Best Things to Do in Atlanta, Georgia: The Ultimate Attractions ...", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.IqznTH1ocAOkn-XVKO4tyQAAAA&pid=Api", "datePublished": "2020-11-23T03:08:00.0000000Z", "contentUrl": "https:\/\/i.pinimg.com\/originals\/f5\/05\/8e\/f5058e44214459ab6506002aee8aa99f.png", "hostPageUrl": "https:\/\/www.pinterest.com\/pin\/101-best-things-to-do-in-atlanta-georgia-the-ultimate-attractions-guide--347621664991790215\/", "contentSize": "897712 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/www.pinterest.com\/pin\/101-best-things-to-do-in-atlanta-georgia-the-ultimate-attractions-guide--347621664991790215\/", "width": 474, "height": 761, "thumbnail": {"width": 474, "height": 761}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=07F2D931B43F714908FF0715A48088951396B542&FORM=IQFRBA", "name": "57 Best &amp; Fun Things To Do In Atlanta (Georgia) - Attractions &amp; Activities", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.zsUlule-ACLeXXDRofPdiAAAAA&pid=Api", "datePublished": "2020-09-12T18:03:00.0000000Z", "contentUrl": "https:\/\/www.busytourist.com\/wp-content\/uploads\/2020\/07\/things-to-do-in-Atlanta-2.jpg", "hostPageUrl": "https:\/\/www.busytourist.com\/things-to-do-in-atlanta-ga\/", "contentSize": "305013 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/www.busytourist.com\/things-to-do-in-atlanta-ga\/", "width": 474, "height": 711, "thumbnail": {"width": 474, "height": 711}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=07F2D931B43F714908FF93753EBBC26AF84A7EE5&FORM=IQFRBA", "name": "57 Best &amp; Fun Things To Do In Atlanta (Georgia) - Attractions &amp; Activities", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.U9xvQMbjg9T0YiriayftJAHaLH&pid=Api", "datePublished": "2020-07-28T16:20:00.0000000Z", "contentUrl": "https:\/\/www.busytourist.com\/wp-content\/uploads\/2020\/07\/Things-To-Do-In-Atlanta-pin1.jpg", "hostPageUrl": "https:\/\/www.busytourist.com\/things-to-do-in-atlanta-ga\/", "contentSize": "306187 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/www.busytourist.com\/things-to-do-in-atlanta-ga\/", "width": 735, "height": 1103, "thumbnail": {"width": 474, "height": 711}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=07F2D931B43F714908FFFE7101020F1CC2A6D241&FORM=IQFRBA", "name": "57 Best &amp; Fun Things To Do In Atlanta (Georgia) - Attractions &amp; Activities", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.H1JPxx3mkr-XvYjeXre46AHaLH&pid=Api", "datePublished": "2020-07-28T16:20:00.0000000Z", "contentUrl": "https:\/\/www.busytourist.com\/wp-content\/uploads\/2020\/07\/best-things-to-do-in-Atlanta-GA-1-2.jpg", "hostPageUrl": "https:\/\/www.busytourist.com\/things-to-do-in-atlanta-ga\/", "contentSize": "337294 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/www.busytourist.com\/things-to-do-in-atlanta-ga\/", "width": 1000, "height": 1500, "thumbnail": {"width": 474, "height": 711}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=602C290BC70BFF7641E4A1A2DAFF6F70A7A3D7A7&FORM=IQFRBA", "name": "The Best Things to Do In Midtown Atlanta - Unexpected Atlanta", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.eKakKX-7lUL-8AAx78alPQHaHa&pid=Api", "datePublished": "2023-04-17T18:24:00.0000000Z", "contentUrl": "https:\/\/unexpectedatlanta.com\/wp-content\/uploads\/2023\/04\/The-Best-Things-to-Do-in-Midtown-Atlanta.png", "hostPageUrl": "https:\/\/unexpectedatlanta.com\/best-things-to-do-midtown-atlanta\/", "contentSize": "210503 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/unexpectedatlanta.com\/best-things-to-do-midtown-atlanta\/", "width": 600, "height": 600, "thumbnail": {"width": 474, "height": 474}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=47F4D7BBCCCEF4B4E79A3775809D838E2B1F77FE&FORM=IQFRBA", "name": "30 Best Things to Do in Atlanta, Georgia", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.dOF_E14DjBhURGyO3gpUPAHaE8&pid=Api", "datePublished": "2022-01-23T13:00:00.0000000Z", "contentUrl": "https:\/\/www.travelandleisure.com\/thmb\/vWy4TaS7bYto1vVS6vmZsrxZzY0=\/1800x1200\/filters:fill(auto,1)\/atlanta-skyline-TODOATL0122-195afa04632e43f5be27ef2dcc94ca3b.jpg", "hostPageUrl": "https:\/\/www.travelandleisure.com\/attractions\/best-things-to-do-in-atlanta-georgia", "contentSize": "442616 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/www.travelandleisure.com\/attractions\/best-things-to-do-in-atlanta-georgia", "width": 1800, "height": 1200, "thumbnail": {"width": 474, "height": 316}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=BE5F128192782DF238FB8C17B09BBC83E3F37F3C&FORM=IQFRBA", "name": "Things To Do in Atlanta | Atlanta Travel Guide - YouTube", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.eFJevefuy_77CA07mDrb7wHaEK&pid=Api", "datePublished": "2020-05-23T04:41:00.0000000Z", "contentUrl": "https:\/\/i.ytimg.com\/vi\/KfyduILpJO8\/maxresdefault.jpg", "hostPageUrl": "https:\/\/www.youtube.com\/watch?v=KfyduILpJO8", "contentSize": "136063 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/www.youtube.com\/watch?v=KfyduILpJO8", "width": 1280, "height": 720, "thumbnail": {"width": 474, "height": 266}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=2319BE605A8B8E34E99F2AFE6D28CFF6F6BEBCBE&FORM=IQFRBA", "name": "Top 50 Things to Do in Atlanta [Infographic]", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.kxVeVo4GNFZJwN_RXwU1AwHaGP&pid=Api", "datePublished": "2019-12-31T16:14:00.0000000Z", "contentUrl": "https:\/\/infographicjournal.com\/wp-content\/uploads\/2019\/12\/Things-to-Do-Atlanta-feat.jpg", "hostPageUrl": "https:\/\/infographicjournal.com\/top-50-things-to-do-in-atlanta\/", "contentSize": "51044 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/infographicjournal.com\/top-50-things-to-do-in-atlanta\/", "width": 800, "height": 674, "thumbnail": {"width": 474, "height": 399}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=D6AB3B605812D6D5E01C093910676D0F8D8DE33F&FORM=IQFRBA", "name": "ALL THE FUN Things to do in Atlanta, Georgia", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.Fe-J-AR_X-7c4ZeUHVaHwQAAAA&pid=Api", "datePublished": "2022-05-22T19:57:00.0000000Z", "contentUrl": "https:\/\/www.acaponeconnection.com\/wp-content\/uploads\/2022\/05\/All-the-Fun-Things-to-Do-in-Atlanta-Georgia-1.jpg", "hostPageUrl": "https:\/\/www.acaponeconnection.com\/all-the-fun-things-to-do-in-atlanta-georgia\/", "contentSize": "87520 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "https:\/\/www.acaponeconnection.com\/all-the-fun-things-to-do-in-atlanta-georgia\/", "width": 474, "height": 316, "thumbnail": {"width": 474, "height": 316}}, {"webSearchUrl": "https:\/\/www.bing.com\/images\/search?q=things+to+do+in+atlanta+on+october+12%2c+2024&id=D9E91667B66F2290E53D96B9F79C0BD5FF864425&FORM=IQFRBA", "name": "Ultimate Atlanta Bucket List (101 Things to Do in Atlanta) » Local ...", "thumbnailUrl": "https:\/\/tse1.mm.bing.net\/th?id=OIP.1mkVJxhMEz5OmxnaoqxJMwHaLD&pid=Api", "datePublished": "2016-02-15T13:10:00.0000000Z", "contentUrl": "http:\/\/localadventurer.com\/wp-content\/uploads\/2016\/02\/101-things-to-do-in-atlanta-bucket-list.jpg", "hostPageUrl": "http:\/\/localadventurer.com\/atlanta-bucket-list-things-to-do-in-atlanta\/", "contentSize": "176335 B", "encodingFormat": "jpeg", "hostPageDisplayUrl": "http:\/\/localadventurer.com\/atlanta-bucket-list-things-to-do-in-atlanta\/", "width": 728, "height": 1086, "thumbnail": {"width": 474, "height": 707}}]}, "relatedSearches": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches", "value": [{"text": "october 2024 atlanta events", "displayText": "october 2024 atlanta events", "webSearchUrl": "https:\/\/www.bing.com\/search?q=october+2024+atlanta+events"}, {"text": "things to do in october atlanta", "displayText": "things to do in october atlanta", "webSearchUrl": "https:\/\/www.bing.com\/search?q=things+to+do+in+october+atlanta"}, {"text": "atlanta 2024 events", "displayText": "atlanta 2024 events", "webSearchUrl": "https:\/\/www.bing.com\/search?q=atlanta+2024+events"}, {"text": "festivals in atlanta 2024", "displayText": "festivals in atlanta 2024", "webSearchUrl": "https:\/\/www.bing.com\/search?q=festivals+in+atlanta+2024"}, {"text": "atlanta fall festivals 2023", "displayText": "atlanta fall festivals 2023", "webSearchUrl": "https:\/\/www.bing.com\/search?q=atlanta+fall+festivals+2023"}, {"text": "downtown atlanta fall festivals", "displayText": "downtown atlanta fall festivals", "webSearchUrl": "https:\/\/www.bing.com\/search?q=downtown+atlanta+fall+festivals"}, {"text": "festivals in atlanta in september", "displayText": "festivals in atlanta in september", "webSearchUrl": "https:\/\/www.bing.com\/search?q=festivals+in+atlanta+in+september"}, {"text": "cool things to do in october", "displayText": "cool things to do in october", "webSearchUrl": "https:\/\/www.bing.com\/search?q=cool+things+to+do+in+october"}]}, "rankingResponse": {"mainline": {"items": [{"answerType": "WebPages", "resultIndex": 0, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.0"}}, {"answerType": "Images", "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#Images"}}, {"answerType": "WebPages", "resultIndex": 1, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.1"}}, {"answerType": "WebPages", "resultIndex": 2, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.2"}}, {"answerType": "WebPages", "resultIndex": 3, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.3"}}, {"answerType": "WebPages", "resultIndex": 4, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.4"}}, {"answerType": "WebPages", "resultIndex": 5, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.5"}}, {"answerType": "WebPages", "resultIndex": 6, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.6"}}, {"answerType": "WebPages", "resultIndex": 7, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.7"}}, {"answerType": "WebPages", "resultIndex": 8, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.8"}}, {"answerType": "WebPages", "resultIndex": 9, "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#WebPages.9"}}, {"answerType": "RelatedSearches", "value": {"id": "https:\/\/api.bing.microsoft.com\/api\/v7\/#RelatedSearches"}}]}}}
```

Here are some events and activities to consider in Atlanta for the next four weekends:

### Weekend of September 21, 2024:
1. [Atlanta Events September 2024: Concerts, Shows, Sports...](https://atlanta-ga.events/september/)
2. [Cool Things To Do in September in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
3. [Atlanta Events - September 2024 - Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)
4. [80+ Fall Festivals & Fairs Around Metro Atlanta](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
5. [The Best Things to Do in Atlanta in September](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/)

### Weekend of September 28, 2024:
1. [Atlanta Events September 2024: Concerts, Shows, Sports...](https://atlanta-ga.events/september/)
2. [Cool Things To Do in September in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
3. [Atlanta Events - September 2024 - Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)
4. [80+ Fall Festivals & Fairs Around Metro Atlanta](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
5. [20 Spectacular Things To Do This September In Atlanta - Secret Atlanta](https://secretatlanta.co/things-to-do-september/)

### Weekend of October 5, 2024:
1. [Atlanta Events - October 2024 - Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
2. [Cool Things To Do in October in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)
3. [Atlanta Events October 2024: Concerts, Shows, Sports...](https://atlanta-ga.events/october/)
4. [80+ Fall Festivals & Fairs Around Metro Atlanta](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
5. [26 Thrilling Things To Do In Atlanta This October - Secret Atlanta](https://secretatlanta.co/things-to-do-october/)

### Weekend of October 12, 2024:
1. [Atlanta Events October 2024: Concerts, Shows, Sports...](https://atlanta-ga.events/october/)
2. [Cool Things To Do in October in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)
3. [80+ Fall Festivals & Fairs Around Metro Atlanta](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
4. [26 Thrilling Things To Do In Atlanta This October - Secret Atlanta](https://secretatlanta.co/things-to-do-october/)
5. [Atlanta Festival Calendar: 2024](https://unexpectedatlanta.com/atlanta-festival-calendar-2024/)

These sources provide a variety of events and activities happening in Atlanta over the next four weekends.


In [20]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;

output.LastMessage.DisplayAs("text/markdown");


Here are some interesting events and activities in Atlanta for the next four weekends:

### September 21, 2024
1. [Adair Park Festival](https://www.atlantaonthecheap.com/fall-festivals-atlanta/): A charming neighborhood festival with plenty of pie, a beer garden, artist market, kid activities, and more. Admission is free, but a pie-tasting pass costs $10.
2. [Indie Craft Experience Fall Market](https://www.fox5atlanta.com/news/things-do-weekend-metro-atlanta-sept-20-22-2024): A one-day craft and vintage market featuring more than 60 vendors at Yaarab Shrine Center.
3. [JapanFest Atlanta](https://discoveratlanta.com/events/main/): A celebration of Japanese culture at the Infinite Energy Center, featuring food, performances, and cultural exhibits.

### September 28, 2024
1. [Sweet Auburn Music Fest](https://discoveratlanta.com/events/main/): Located on Auburn Avenue, this festival offers live music from R&B to gospel, an international food court, national artists, celebrities, and vendors of all types.
2. [Carter Presidential Library & Museum](https://www.atlantaonthecheap.com/september-things-to-do-atlanta/): Free admission in honor of President Jimmy Carter's 100th birthday.
3. [Atlanta Streets Alive](https://www.atlantaonthecheap.com/fall-festivals-atlanta/): Streets temporarily closed to traffic for walking, running, biking, and other activities. 

### October 5, 2024
(Please note that no specific events for this date were found. Here are some ongoing activities you might consider.)
1. [Grant Park Past & Future Food Walk](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/): A fun walking tour showcasing the best of Atlanta's food and history.
2. [Atlanta Food & Wine Festival](https://www.experism.com/atlanta/september): Highlights the best food, wine, beer, and cocktails Atlanta has to offer.

### October 12, 2024
(Please note that no specific events for this date were found. Here are some ongoing activities you might consider.)
1. [Explore Atlanta's Fall Festivals](https://www.atlantaonthecheap.com/fall-festivals-atlanta/): Various fall festivals and fairs around metro Atlanta.
2. [Experience Atlanta's Best Eats](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/): Try some of the best eats in Atlanta on various food tours.

For more details on each event, you can visit the provided links. Enjoy your time in Atlanta!

## Clean Up

In [12]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search Test");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
